# Imports

In [ ]:
import pandas as pd
import os
pd.set_option('display.max_columns', None)

In [ ]:
# notice that they have 'target_gender' or 'target_race' prefix, but we'll replace them later
identities = ['target_gender_men', 'target_gender_women','target_gender_transgender', 'target_race_white', 'target_race_black', 'target_race_asian', ]
categories = ['men vs women', 'men vs transgender', 'white vs black', 'white vs asian']
figsize = (14,8)
epsilon_list = [0.5, 1.0, 3.0, 6.0, 9.0]
dataset_name = 'ucberkeley'
dataset_directory = f'../../results/{dataset_name}/'
model_name = 'bert-base-uncased'
output_folder = f'{dataset_directory}/average/{model_name}'
runs = 3
linewidth = 3.5
markersize = 13
markers = ['o', 'v', 's', 'x',  'h', 'D', '^', '>', 'p', '<', '*', 'P',  '.', '+']

if not os.path.exists(output_folder):
    os.makedirs(output_folder, exist_ok=True)

# Average bias and overall results

## Load bias and results

In [ ]:
bias_df_dict = {}
overall_results_dict = {}

for run in range(1, runs+1):
    run_folder = f'{dataset_directory}/run {run}'
    model_folder = os.path.join(run_folder, model_name)

    normal_folder = os.path.join(model_folder, 'normal')
    bias_df = pd.read_csv(os.path.join(normal_folder, 'bias.csv'))
    overall_results = pd.read_csv(os.path.join(normal_folder, 'overall_results.csv'))
    if run ==1:
        bias_df_dict['None'] = [bias_df]
        overall_results_dict['None'] = [overall_results]
    else:
        bias_df_dict['None'].append(bias_df)
        overall_results_dict['None'].append(overall_results)

    for epsilon in epsilon_list:
        dp_folder = os.path.join(model_folder, f'epsilon {epsilon}')
        
        bias_df = pd.read_csv(os.path.join(dp_folder, 'bias.csv'))
        overall_results = pd.read_csv(os.path.join(dp_folder, 'overall_results.csv'))

        if run ==1:
            bias_df_dict[epsilon] = [bias_df]
            overall_results_dict[epsilon] = [overall_results]
        else:
            bias_df_dict[epsilon].append(bias_df)
            overall_results_dict[epsilon].append(overall_results)

## Concatenate. Dump average

In [ ]:
for epsilon in ['None'] + epsilon_list:
    bias_df_dict[epsilon] = pd.concat(bias_df_dict[epsilon])
    # mean = bias_df_dict[epsilon].groupby('fairness_metrics').aggregate('mean').reset_index()
    # mean.round(3).to_csv(
    #     os.path.join(output_folder, f'bias_epsilon_{epsilon}.csv'), 
    #     index=False
    # )

    overall_results_dict[epsilon] = pd.concat(overall_results_dict[epsilon])
    # mean = overall_results_dict[epsilon].groupby('metrics').aggregate('mean').reset_index()
    # mean.round(3).to_csv(
    #     os.path.join(output_folder, f'overall_results_epsilon_{epsilon}.csv'), 
    #     index=False
    # )

# Plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import StrMethodFormatter
# Apply the default theme
sns.set_theme()
sns.set(font_scale = 2)

import matplotlib as mpl
mpl.rcParams['lines.markersize'] = markersize
mpl.rcParams['lines.linewidth'] = linewidth

formatter = StrMethodFormatter('{x:,.2f}')

## Boxplot

In [ ]:
dictionary = {
    'Subgroup': [],
    'Metric': [],
    'Train Type': []
}
metric= 'EqOdd'

def key_map(key):
    if key=='None':
        return 'Non-DP'
    return f'ε <={key}'

for category in categories:
    for key in bias_df_dict.keys():
        bias_df = bias_df_dict[key]
        values = bias_df[bias_df['fairness_metrics']==metric][category].values
        dictionary['Metric'].extend(values)
        dictionary['Subgroup'].extend([category]*len(values))
        dictionary['Train Type'].extend([key_map(key)]*len(values))

df = pd.DataFrame(dictionary)

In [ ]:
hue_order = df['Train Type'].unique()
hue_order = [hue_order[0]] + sorted(hue_order[1:], reverse=True)
hue_order

In [ ]:
# this returns the df with an increasing privacy order of Train type
def order(df):
    return df.set_index('Train Type').reindex(hue_order).reset_index()

In [ ]:
plt.figure(figsize=figsize)
sns.boxplot(x = 'Subgroup', y = 'Metric', hue='Train Type', hue_order=hue_order, data = df, linewidth=2.5)
plt.xlabel('')
plt.ylabel('')
plt.gca().yaxis.set_major_formatter(formatter)
plt.tight_layout()
# plt.savefig(os.path.join(output_folder, 'eodds_boxplot.pdf'))

## Overall result

In [ ]:
total = []
for epsilon in ['None']+epsilon_list:
    mean = overall_results_dict[epsilon].groupby('metrics').aggregate('mean').reset_index()
    mean['Train Type'] = key_map(epsilon)
    total.append(mean)

In [ ]:
total_df = pd.concat(total)
total_df.rename({col:'_'.join(col.split('_')[2:]) for col in total_df.columns if 'target_' in col}, axis=1, inplace=True)
identities = ['_'.join(identity.split('_')[2:]) for identity in identities]
total_df.head()

In [ ]:
total_df[total_df['metrics'].isin(['accuracy', 'f1_score', 'auc'])][['metrics', 'Total', 'Train Type']].round(3)

### Total dataset

In [ ]:
selected_metrics = ['auc', 'f1_score', 'accuracy', 'precision', 'recall']
plt.figure(figsize=figsize)

for i, selected_metric in enumerate(selected_metrics):
    selected_df = total_df[total_df['metrics']==selected_metric].round(3)
    selected_df = order(selected_df)
    sns.lineplot(x=selected_df["Train Type"], y=selected_df["Total"], label=selected_metric, marker=markers[i])

plt.legend(loc='upper right', fancybox=True, framealpha=0.4)
plt.tight_layout()
plt.gca().yaxis.set_major_formatter(formatter)
plt.savefig(os.path.join(output_folder, 'overall_total.pdf'))
plt.show()

### Util plot_overall_metric

In [ ]:
def plot_overall_metric(selected_metric, title=None, figure_name=None):
    plt.figure(figsize=figsize)
    selected_df = total_df[total_df['metrics']==selected_metric]
    selected_df = order(selected_df)

    for i, identity in enumerate(identities):
        plt.plot(selected_df["Train Type"], selected_df[identity], label=identity, marker=markers[i])

    plt.legend(loc='upper right', fancybox=True, framealpha=0.4)

    # https://stackoverflow.com/questions/29188757/matplotlib-specify-format-of-floats-for-tick-labels
    plt.gca().yaxis.set_major_formatter(formatter)
    if title:
        plt.title(title)
    plt.tight_layout()
    if figure_name:
        plt.savefig(os.path.join(output_folder, figure_name))
    plt.show()

### BNSP

In [ ]:
plot_overall_metric('bnsp_auc', figure_name='bnsp.pdf')

### BPSN

In [ ]:
plot_overall_metric('bpsn_auc', figure_name= 'bpsn.pdf')

### Subgroup auc

In [ ]:
plot_overall_metric('auc', figure_name='roc_auc.pdf')

### F1-score

In [ ]:
plot_overall_metric('f1_score', figure_name= 'f1_score.pdf')

### False positive rate

In [ ]:
plot_overall_metric('false positive rate', figure_name= 'false_positive_rate.pdf')

### Precision

In [ ]:
plot_overall_metric('precision', figure_name= 'precision.pdf')

### Recall

In [ ]:
plot_overall_metric('recall', figure_name= 'recall.pdf')

## Bias

In [ ]:
total = []
for epsilon in ['None']+epsilon_list:
    mean = bias_df_dict[epsilon].groupby('fairness_metrics').aggregate('mean').reset_index()
    mean['Train Type'] = key_map(epsilon)
    total.append(mean)

total_df = pd.concat(total)
total_df.head()

### Util plot_bias_metric

In [ ]:
def plot_bias_metric(category, title=None, figure_name= None):
    plt.figure(figsize=figsize)
    metric_column = 'fairness_metrics'
    # selected_metrics = total_df[metric_column].unique()
    selected_metrics = ['EqOdd','EqOpp1', 'EqOpp0',  'parity', 'p-accuracy', 'up-accuracy']
    
    for i, selected_metric in enumerate(selected_metrics):
        selected_df = total_df[total_df[metric_column]==selected_metric]
        selected_df = order(selected_df)
        
        plt.plot(selected_df["Train Type"], selected_df[category], label=selected_metric, marker=markers[i])

    plt.legend(loc='upper right', fancybox=True, framealpha=0.5)

    plt.gca().yaxis.set_major_formatter(formatter)
    if title:
        plt.title(title)
    plt.tight_layout()
    if figure_name:
        plt.savefig(os.path.join(output_folder, figure_name))

    plt.show()

### Race

In [ ]:
for category in ['white vs black', 'white vs asian']:
    plot_bias_metric(category, figure_name=f"racial_bias_{'_'.join(category.split())}.pdf")

### Gender

In [ ]:
for category in ['men vs women', 'men vs transgender']:
    plot_bias_metric(category, figure_name=f"gender_bias_{'_'.join(category.split())}.pdf")